# LBB: Building LLM Applications for Structured Data Insights Use RAG

Muh Amri Sidiq

## Introduction

🔍 What is an LLM (Large Language Model)?
A Large Language Model is an advanced AI system trained on massive amounts of text data. It understands and generates human-like language, enabling it to:
 - Answer questions
 - Write and summarize content
 - Support dialogue and reasoning
 - Assist in data analysis, and more.

LLMs include models like OpenAI’s GPT, Meta’s LLaMA, or Google’s Gemini.

🔎 What is RAG (Retrieval-Augmented Generation)?
Retrieval-Augmented Generation (RAG) is a framework that combines:
 - Retrieval: Searching for relevant information from external data (e.g., documents, databases, company reports)
 - Generation: Using an LLM to generate answers based on that retrieved data.

📌 When Do You Need RAG?
You should consider using RAG when:
 - The LLM doesn't know your private/internal data (like sales reports, product records, etc.)
 - You need fact-based or evidence-grounded responses, not just general knowledge

💼 Example: RAG with the US E-commerce Record 2020 Dataset
Using the dataset from Kaggle: US E-commerce Record 2020, you can:
 - Let the LLM answer questions about customer trends, sales performance, and product behavior
 - Ensure that responses are based on actual transaction data instead of generic e-commerce knowledge
 - Build a chatbot or dashboard that answers queries like:

   - “What was the average order value for returning customers in Texas?”
   - “List the top 5 most sold product categories in December 2020.”
   - “How does sales performance differ between payment types?”

This allows business users to interact with internal data conversationally, while ensuring accuracy and contextual relevance through retrieval.

## Workflow RAG

there are several steps to determine the RAG flow

### 1. Read Data

the data we use is tabular data that contains ecomerce transaction columns.

In [1]:
import pandas as pd

# Read the e-commerce dataset
# The dataset is assumed to be in the same directory as this script.
ecommerce = pd.read_csv("data/US  E-commerce records 2020.csv", encoding='cp1252')
ecommerce.head()

,Order Date,Row ID,Order ID,Ship Mode,Customer ID,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,01-01-20,849,CA-2017-107503,Standard Class,GA-14725,Consumer,United States,Lorain,Ohio,44052,East,FUR-FU-10003878,Furniture,Furnishings,"Linden 10"" Round Wall Clock, Black",48.896,4,0.2,8.5568
1,01-01-20,4010,CA-2017-144463,Standard Class,SC-20725,Consumer,United States,Los Angeles,California,90036,West,FUR-FU-10001215,Furniture,Furnishings,"Howard Miller 11-1/2"" Diameter Brentwood Wall ...",474.430,11,0.0,199.2606
2,01-01-20,6683,CA-2017-154466,First Class,DP-13390,Home Office,United States,Franklin,Wisconsin,53132,Central,OFF-BI-10002012,Office Supplies,Binders,Wilson Jones Easy Flow II Sheet Lifters,3.600,2,0.0,1.7280
3,01-01-20,8070,CA-2017-151750,Standard Class,JM-15250,Consumer,United States,Huntsville,Texas,77340,Central,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,454.560,5,0.2,-107.9580
4,01-01-20,8071,CA-2017-151750,Standard Class,JM-15250,Consumer,United States,Huntsville,Texas,77340,Central,FUR-FU-10002116,Furniture,Furnishings,"Tenex Carpeted, Granite-Look or Clear Contempo...",141.420,5,0.6,-187.3815


In [2]:
ecommerce.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3312 entries, 0 to 3311
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Order Date    3312 non-null   object 
 1   Row ID        3312 non-null   int64  
 2   Order ID      3312 non-null   object 
 3   Ship Mode     3312 non-null   object 
 4   Customer ID   3312 non-null   object 
 5   Segment       3312 non-null   object 
 6   Country       3312 non-null   object 
 7   City          3312 non-null   object 
 8   State         3312 non-null   object 
 9   Postal Code   3312 non-null   int64  
 10  Region        3312 non-null   object 
 11  Product ID    3312 non-null   object 
 12  Category      3312 non-null   object 
 13  Sub-Category  3312 non-null   object 
 14  Product Name  3312 non-null   object 
 15  Sales         3312 non-null   float64
 16  Quantity      3312 non-null   int64  
 17  Discount      3312 non-null   float64
 18  Profit        3312 non-null 

### 2. Data to Text Transformation

So that the data can be used, which was originally tabular, it must first be converted into text form, so that LLM can read it. By combining all the columns of each row, the first step is to make the data of each row into 1 full sentence as a condition for LLM to read it.

In [3]:
# membuat sebuah fungsi
def merge_column(df, column_data):
    df['teks'] = df[column_data].astype('str').agg(' | '.join, axis = 1)
    return df

In [4]:
cols = ['Order Date', 'Row ID', 'Order ID', 'Ship Mode', 'Customer ID',
        'Segment', 'Country', 'City', 'State', 'Postal Code',
        'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name',
        'Sales', 'Quantity', 'Discount', 'Profit']

merge_column(df=ecommerce, column_data=cols)
ecommerce.head(5)

,Order Date,Row ID,Order ID,Ship Mode,Customer ID,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,teks
0,01-01-20,849,CA-2017-107503,Standard Class,GA-14725,Consumer,United States,Lorain,Ohio,44052,East,FUR-FU-10003878,Furniture,Furnishings,"Linden 10"" Round Wall Clock, Black",48.896,4,0.2,8.5568,01-01-20 | 849 | CA-2017-107503 | Standard Cla...
1,01-01-20,4010,CA-2017-144463,Standard Class,SC-20725,Consumer,United States,Los Angeles,California,90036,West,FUR-FU-10001215,Furniture,Furnishings,"Howard Miller 11-1/2"" Diameter Brentwood Wall ...",474.430,11,0.0,199.2606,01-01-20 | 4010 | CA-2017-144463 | Standard Cl...
2,01-01-20,6683,CA-2017-154466,First Class,DP-13390,Home Office,United States,Franklin,Wisconsin,53132,Central,OFF-BI-10002012,Office Supplies,Binders,Wilson Jones Easy Flow II Sheet Lifters,3.600,2,0.0,1.7280,01-01-20 | 6683 | CA-2017-154466 | First Class...
3,01-01-20,8070,CA-2017-151750,Standard Class,JM-15250,Consumer,United States,Huntsville,Texas,77340,Central,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,454.560,5,0.2,-107.9580,01-01-20 | 8070 | CA-2017-151750 | Standard Cl...
4,01-01-20,8071,CA-2017-151750,Standard Class,JM-15250,Consumer,United States,Huntsville,Texas,77340,Central,FUR-FU-10002116,Furniture,Furnishings,"Tenex Carpeted, Granite-Look or Clear Contempo...",141.420,5,0.6,-187.3815,01-01-20 | 8071 | CA-2017-151750 | Standard Cl...


### 3. Search for Text Similarities

#### 3.1 Embedding Teks

At this stage we convert the text into a vector representation capturing the meaning of the text. In this case looking for similarities between one data and another.

Information about the similarity between one data and another is important because the initial goal of RAG is to retrieve data information that is relevant to the given question, in order to generate relevant LLM answers.

In [5]:
# prepare the embedding model to be used
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

`query =  "what segment makes the most purchases in the state of California?"`

`embedding_query_new = model.encode(query, convert_to_numpy=True)`

Setting up a query and embedding it for the information retrieval process.
The purpose of this code is to convert the text question ("query"). into a vector representation (embedding) so that it can be compared with data already indexed by FAISS. With this representation, the system can search for the most relevant rows of data based on similarity of meaning.

In [6]:
# prepare query & embedding new
query =  "what segment makes the most purchases in the state of California?"
embedding_query_new = model.encode(query, convert_to_numpy=True)

`embedding_dataframe = model.encode(ecommerce['teks'], convert_to_numpy=True)`

This code serves to convert the text in the text column of the ecommerce DataFrame into a vector representation (embedding) using the SentenceTransformer model. This embedding is important for performing semantic search with FAISS
 - model.encode(...): Calls the model to convert the text to embedding.
 - ecommerce[‘text’]: A column containing combined information from multiple
 - columns, prepared beforehand with a function like transform_data.

convert_to_numpy=True: Sets the output to be in numpy array format, which is required for use in FAISS index or other mathematical operations. The main purpose of this step is to prepare the data so that it can be searched based on similarity of meaning, not just keyword matching.

In [7]:
# prepare embedding for our data
embedding_dataframe = model.encode(ecommerce['teks'], convert_to_numpy=True)

`cosine_scores = util.cos_sim(embedding_query_new, embedding_dataframe)`

This code is used to calculate the semantic similarity between the query and all data in the embedding form.

Details:
 - embedding_query_new: The representation vector of the query that has been converted to embedding.
 - embedding_dataframe: Set of embeddings of all text data in the text column of the DataFrame.
 - util.cos_sim(...): A function from sentence_transformers.util that calculates cosine similarity, a measure of how similar two direction vectors are in multidimensional space.

Main objective: Find out how relevant each line of data is to the user's question based on closeness of meaning, not just word similarity.

Output:
 - cosine_scores produces a matrix with values between -1 and 1.
 - A value close to 1 means very similar, close to 0 means not similar, and a negative value means very different in semantic direction.

In [8]:
# calculate cosine similarity
cosine_scores = util.cos_sim(embedding_query_new, embedding_dataframe)
cosine_scores

tensor([[0.1494, 0.2476, 0.0978,  ..., 0.3054, 0.3561, 0.2677]])

#### 3.2: FAISS Indexing From Question

`import faiss, import numpy as np` This line imports two important libraries:
 - `faiss`: Used for indexing and similarity-based vector search.
 - `numpy`: Used for numeric array manipulation, such as vector normalization.

`embedding_dataframe = embedding_dataframe / np.linalg.norm(embedding_dataframe, axis=1, keepdims=True)`
`embedding_dataframe = embedding_dataframe.astype('float32')`
Manual normalization of embedding
 - Normalization is done so that each embedding vector has a length (norm) = 1.
 - This is important because FAISS uses inner product (dot product) to calculate similarity, and when vectors are normalized, dot product = cosine similarity
 - astype(‘float32’) is required because FAISS only accepts float32 data type.
   
   🔍 This normalization ensures the similarity search in FAISS is equivalent to cosine similarity.

`dimension = embedding_dataframe.shape[1]` Specifying Embedding Dimensions
 - Retrieves the number of dimensions of the embedding (for example, 384 if using the 'paraphrase-MiniLM-L6-v2' model).
 - This value is needed when creating the FAISS index:
    This dimension is the size of the vector space where all embeddings are placed.

In [9]:
import faiss
import numpy as np

# Calculate the cosine similarity scores with manually
embedding_dataframe = embedding_dataframe / np.linalg.norm(embedding_dataframe, axis=1, keepdims=True)
embedding_dataframe = embedding_dataframe.astype('float32')

# take value embedding
dimension = embedding_dataframe.shape[1]
dimension

384

`index = faiss.IndexFlatL2(dimension)` Creating and Populating the FAISS Index

This line makes the FAISS index use L2 (Euclidean) distance as the similarity metric.
Explanation:
 - IndexFlatL2 is a FAISS index type that calculates the Euclidean distance (L2 distance) between vectors.
 - dimension is the number of dimensions of each embedding vector (e.g. 384).
 - This index does not use fast search structures (such as IVF or HNSW) - suitable for small to medium datasets.

   📌 Use IndexFlatIP if you want to calculate cosine similarity with normalized vectors.

`index.add(embedding_dataframe)` This line adds all embedding vectors to the FAISS index.
Explanation:
 - embedding_dataframe contains all the encoded vectors of the text in the dataset.
 - Once added, the index is ready to be used for similarity-based queries.

   FAISS can now be used to find the most similar data to the given query.

In [10]:
index = faiss.IndexFlatL2(dimension)
index.add(embedding_dataframe)

The dimension results if compared back to the example query question above, will be as below.

In [11]:
query

'what segment makes the most purchases in the state of California?'

`embedding_index_query = model.encode([query])`

Explanation:
 - This code is used to convert a user's query into an embedding vector, so that it can be compared with vectors of previously indexed data.
 - `model.encode(...)`: A function of the SentenceTransformer that converts text into a numerical representation (embedding).
 - `[query]`: Given in list form because the model expects the input to be a list of strings, even if it is only one query.
 - The result is a 2-dimensional array (shape: [1, dimension]), which is the format FAISS needs for the search process.
 
 Main objective: Convert the user's question into a vector format, so that it can be compared against the entire data to find the most relevant answer.

In [12]:
embedding_index_query = model.encode([query])

`D, I = index.search(embedding_index_query, k = 2)`

Explanation:
 - This code is used to find the k closest (most similar) vectors in the FAISS index to the query that has been converted to embedding.
 - `embedding_index_query`: The encoding result vector of the user query.
 - `k = 2`: Retrieve the 2 closest results (top-2 most similar).
 - `index.search(...)`: FAISS function to perform a distance-based search (e.g. Euclidean if using IndexFlatL2, or cosine if using IndexFlatIP + normalization).

Output:
 - `D`: The distance matrix or similarity score between the query and the k closest results.
 - `I`: The row index matrix of the original data that is most similar to the query.

Main objective: Find the rows of data in the dataset that are most relevant to the user's query based on embedding.

In [13]:
D, I = index.search(embedding_index_query, k = 2)

`D:` the distance between the query_vector and the nearest vectors.

In [14]:
D

array([[44.061226, 44.409435]], dtype=float32)

`I:` index of most similar vectors

In [15]:
I

array([[3178,  411]], dtype=int64)

In [16]:
cosine_scores

tensor([[0.1494, 0.2476, 0.0978,  ..., 0.3054, 0.3561, 0.2677]])

Function: Build FAISS Index with Cosine Similarity

Function Explanation:
The build_faiss_index_cosine(text) function is used to create a FAISS index based on cosine similarity, which is useful in the process of searching relevant data based on the similarity of text meaning.

Steps:
1. Embedding Text: `embedding = model.encode(teks , convert_to_numpy=True)`
   Convert a text list (text) into a numeric vector (embedding) using SentenceTransformer.
2. Normalization for Cosine Similarity: `embedding = embedding / np.linalg.norm(embedding, axis=1, keepdims=True)`,
`embedding = embedding.astype('float32')`. 
   - Normalize each vector to unit-norm so that dot product = cosine similarity.  - Convert to float32 as FAISS only supports this data type.
3. Create and Fill Index:
   `index = faiss.IndexFlatL2(dimension)`
    `index.add(embedding)`
   - Creating FAISS index is based on L2 distance, but since the vectors are already normalized, the L2 search is equivalent to cosine similarity.
   - dimension is the number of dimensions of the embedding vector (e.g. 384 or 768 depending on the model).
4. Return:
   - Returns the FAISS index and its embedding for use in the search process.

Notes:
- FAISS does not provide IndexFlatCosine function, so to do cosine similarity, we need to do manual normalization and still use IndexFlatL2.
- Make sure the input text is in the form of a list of strings so that the encode runs smoothly.

The main purpose of this function: Build a meaning-based fast search structure for use in RAG (Retrieval-Augmented Generation) or data-driven question and answer systems.



In [17]:
# make function to build FAISS index with cosine similarity
def build_faiss_index_cosine(teks):
    # Section for performing embeddings
    embedding = model.encode(teks , convert_to_numpy=True)

    # Perform cosine calculation
    embedding = embedding / np.linalg.norm(embedding, axis=1, keepdims=True)
    embedding = embedding.astype('float32')

    # Indexing
    dimension = embedding.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embedding)

    return index, embedding

Function: Retrieve Relevant Data from FAISS Index

Function Explanation:
The retrieve() function is used to retrieve the most relevant rows of data against a query based on meaning similarity using the FAISS index.

Detailed Steps:
1. Encode and Normalize Query:

   `query_embedding = model.encode([query], convert_to_numpy=True)`
   `query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)`
   `query_embedding = query_embedding.astype("float32")`

   - The query is converted into a vector (embedding).
   - The query vector is normalized so that it can be compared with the embedding data based on cosine similarity.
   - Conversion to float32 as FAISS only supports this format.
2. Search the FAISS Index:
   
   `scores, indices = index.search(query_embedding, top_k)`

   - Searches the index to find the top_k most similar results.
   - scores: Similarity or distance values (higher if cosine similarity).
   - indices: Index of rows from the original data that are most relevant.
3. Fetch Data from DataFrame:
   
   `result_df = df.iloc[indices[0]].copy()`
   `result_df['similarity_score'] = scores[0]`

   - Retrieve rows from the original DataFrame (df) based on the FAISS result.
   - index.- Adds a new column similarity_score to display the similarity score.
4. Return Results:
   
   `return result_df`

   - Returns a DataFrame containing relevant data rows and similarity scores.

Final Destination:
This function is the core part of the RAG (Retrieval-Augmented Generation) system, which enables the model to answer the most relevant data-driven questions quickly and accurately.


In [18]:
# make function to retrieve data from FAISS index
def retrieve(query, index, df, top_k=3):
    # 1. Encode dan normalization query
    query_embedding = model.encode([query], convert_to_numpy=True)
    query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)
    query_embedding = query_embedding.astype("float32")

    # 2. Search to FAISS
    scores, indices = index.search(query_embedding, top_k)

    # 3. Retrieve the data row according to the indexing result
    result_df = df.iloc[indices[0]].copy()
    result_df['similarity_score'] = scores[0]

    return result_df

Function: Generate Answers Using OpenAI GPT

Function Explanation:
The generate_answer() function is used to generate text-based answers using the OpenAI ChatCompletion API (GPT-4.1-mini model). It combines the user's question and relevant context data to generate an informative and specific answer.

 Line-by-Line Explanation:
1. Set the OpenAI API Key:
   `openai.api_key = api_key`

   The API key is set so that the request can be authenticated and the OpenAI service can be used.
2. Set up a Prompt System:

   `system_message = "Kamu adalah asisten cerdas yang menjawab pertanyaan berdasarkan data yang diberikan."`
   
   System messages to direct the model to act as a data-driven intelligent assistant.
3. Compose Prompts from Users:
   
   `user_message = f"""`
   `Pertanyaan: {query}`

   `Data yang relevan:`

   `{context}`
   
   `"""`

   Combining questions and relevant data (results from FAISS retrieval) into a prompt format for the model.

4. Send Request to OpenAI:
   
   `response = openai.ChatCompletion.create(`

    `model="gpt-4.1-mini",`
    
    `messages=[...],`
    
    `temperature=0.3,`
    
    `max_tokens=1000`
    
    `)`

    - Model: The model used (in this case GPT-4.1-mini).
    - messages: A list of messages to set the context of the conversation. 
    - temperature=0.3: Creativity control. The lower it is, the more deterministic the output.
    - max_tokens=1000: Limit on the length of the generated answer.

5. Returning Answers:
   
   `return response.choices[0].message["content"]`

   Retrieve the answer from the result provided by OpenAI and return it as text.

Function Objective:
Connecting the data retrieval process with GPT's reasoning capabilities, so that the system can answer questions based on semantically discovered information from the dataset.




In [19]:
import openai

def generate_answer(query, context, api_key):
    # to enter the api key of the generative model being used
    openai.api_key = api_key
    # to tell you specifically what the generative model needs to do.
    system_message = "Kamu adalah asisten cerdas yang menjawab pertanyaan berdasarkan data yang diberikan."
    # for users to input questions or data that they want to learn.
    user_message = f"""
    Pertanyaan: {query}

    Data yang relevan:
    {context}
    """
    response = openai.ChatCompletion.create(
        model="gpt-4.1-mini", # the model being used
        # system messages or to process the input data or user
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        # to adjust the level of randomness in the next word prediction
        temperature=0.3,
        # to set the maximum number of tokens that can be processed
        max_tokens=1000
    )
    return response.choices[0].message["content"]

Generate Answers Using GPT Based on Relevant Data

Explanation:
The generate_answer() function is called to answer a user-specific question based on the data set provided in the text fields of the ecommerce DataFrame.
Parameters:
 - `query`:
   The user question that the model wants to answer.
   In this example:
   “what segment makes the most purchases in the state of California?”

- `context`:
   A set of relevant data in text form (usually the result of multiple columns combined), used as context for the GPT model to answer based on real information.
   Here: ecommerce[‘text’].

- `api_key`:
  The API key used to access the OpenAI ChatCompletion API. Must be provided for the request to run.

In [ ]:
generate_answer(query= "What is the total sales for State = CA, Ship Mode = First Class, and Segment = Home Office?",
                context=  ecommerce['teks'],  
                api_key= "")

'Data yang diberikan hanya berupa sebagian kecil dari dataset dan tidak mencakup informasi lengkap mengenai kolom-kolom seperti Ship Mode, Segment, dan nilai Sales secara eksplisit. Oleh karena itu, saya tidak dapat menghitung total penjualan (total sales) untuk kondisi State = CA, Ship Mode = First Class, dan Segment = Home Office hanya berdasarkan data yang tersedia.\n\nJika Anda dapat menyediakan data lengkap yang mencakup kolom State, Ship Mode, Segment, dan nilai Sales, saya dapat membantu menghitung total penjualan sesuai kriteria tersebut.'

## Conclusion

Through a series of code builds, we successfully implemented an intelligent and interactive Retrieval-Augmented Generation (RAG) system, capable of answering user queries based on 2020 e-commerce data. The system combines the strengths of:

Key Technical Components:
1. Data Preprocessing & Transformation
   - E-commerce data is read from CSV files with special encoding (cp1252) and combined into a single text column (text) for semantic analysis.

2. Embedding & Indexing
   - Using SentenceTransformer model (paraphrase-MiniLM-L6-v2) to transform text into embedding vectors.
   - Embedding is normalized to simulate cosine similarity using FAISS (IndexFlatL2).
   - FAISS is used to build the vector index and perform a quick search for meaning similarity.

2. Semantic Retrieval
   - User queries are converted into embedding vectors and matched against the index to find the most relevant (top-k) data.
   - The most similar data rows are returned with a similarity score.

3. Natural Language Answers (LLM)
   - Using OpenAI GPT-4.1-mini, the system forms natural answers based on the context of the found data.
   - Prompts consist of system messages and user messages that include the question and context of the retrieval results.

4. Interactive Interface (UI)
   - With Streamlit, users can upload CSV files, select fields, type questions, and view answers directly in one application page.

Benefits and Objectives
This system enables:
 - Data-driven question answering without having to perform SQL queries or manual exploration.
 - Easy integration with new datasets by simply uploading CSV files.
 - Combining the power of search engine (FAISS) with generative reasoning (GPT), ideal for business insight, customer analytics, and large text analysis.

Next Steps (Optional)
 - Add answer highlights or traceback.
 - Implement data filtering before indexing (e.g. only rows from California). 
 - Implement query result storage or cache with Streamlit session.

With this system, you have built a solid foundation for an AI assistant based on internal company data - extendable to HR data, finance, logistics, or even other free text reports.